In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras.layers import ConvLSTM2D, BatchNormalization, Conv2D, UpSampling3D, MaxPooling3D, Dropout
from tensorflow.keras.optimizers import Adam,SGD
from tensorflow.keras import regularizers
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

In [2]:
from tensorflow.keras import backend
backend.set_image_data_format('channels_first')

In [3]:
def get_unet_LSTM():
    #concat_axis = 3 # 3 o 1
    concat_axis = 2 
    #inputs = layers.Input(shape = (80, 120, 3))
    inputs = layers.Input(shape = (4, 3, 128, 144))
    
    #encoder
    bn0 = BatchNormalization(axis=2)(inputs)
    conv1 = layers.ConvLSTM2D(32, (3, 3), activation='relu', padding='same',recurrent_activation = "hard_sigmoid",return_sequences=True,name='conv1_1')(bn0)
    bn1 = BatchNormalization(axis=2)(conv1)
    conv1 = layers.ConvLSTM2D(32, (3, 3), activation='relu', padding='same',recurrent_activation = "hard_sigmoid",return_sequences=True)(bn1)
    bn2 = BatchNormalization(axis=2)(conv1)
    pool1 = layers.MaxPooling3D(pool_size=(1,2, 2))(bn2)
    conv2 = layers.ConvLSTM2D(64, (3, 3), activation='relu', padding='same', recurrent_activation = "hard_sigmoid",return_sequences=True)(pool1)
    bn3 = BatchNormalization(axis=2)(conv2)
    conv2 = layers.ConvLSTM2D(64, (3, 3), activation='relu', padding='same', recurrent_activation = "hard_sigmoid",return_sequences=True)(bn3)
    bn4 = BatchNormalization(axis=2)(conv2)
    pool2 = layers.MaxPooling3D(pool_size=(1, 2, 2))(bn4)

    conv3 = layers.ConvLSTM2D(128, (3, 3), activation='relu', padding='same',recurrent_activation = "hard_sigmoid",return_sequences=True)(pool2)
    bn5 = BatchNormalization(axis=2)(conv3)
    conv3 = layers.ConvLSTM2D(128, (3, 3), activation='relu', padding='same', recurrent_activation = "hard_sigmoid",return_sequences=True)(bn5)
    bn6 = BatchNormalization(axis=2)(conv3)
    pool3 = layers.MaxPooling3D(pool_size=(1, 2, 2))(bn6)

    conv4 = layers.ConvLSTM2D(256, (3, 3), activation='relu', padding='same', recurrent_activation = "hard_sigmoid",return_sequences=True)(pool3)
    bn7 = BatchNormalization(axis=2)(conv4)
    conv4 = layers.ConvLSTM2D(256, (3, 3), activation='relu', padding='same', recurrent_activation = "hard_sigmoid",return_sequences=True)(bn7)
    bn8 = BatchNormalization(axis=2)(conv4)
    #pool4 = layers.MaxPooling2D(pool_size=(2, 3))(bn8)
    pool4 = layers.MaxPooling3D(pool_size=(1, 2, 2))(bn8)
    
    conv5 = layers.ConvLSTM2D(512, (3, 3), activation='relu', padding='same', recurrent_activation = "hard_sigmoid",return_sequences=True)(pool4)
    bn9 = BatchNormalization(axis=2)(conv5)
    conv5 = layers.ConvLSTM2D(512, (3, 3), activation='relu', padding='same', recurrent_activation = "hard_sigmoid",return_sequences=True)(bn9)
    bn10 = BatchNormalization(axis=2)(conv5)

    ##decoder
    #up_conv5 = layers.UpSampling2D(size=(2, 3))(bn10)
    up_conv5 = layers.UpSampling3D(size=(1,2, 2))(bn10)
    up6 = layers.concatenate([up_conv5, conv4], axis=concat_axis)
    conv6 = layers.ConvLSTM2D(256, (3, 3), activation='relu', padding='same', recurrent_activation = "hard_sigmoid",return_sequences=True)(up6)
    bn11 = BatchNormalization(axis=2)(conv6)
    conv6 = layers.ConvLSTM2D(256, (3, 3), activation='relu', padding='same', recurrent_activation = "hard_sigmoid",return_sequences=True)(bn11)
    bn12 = BatchNormalization(axis=2)(conv6)

    up_conv6 = layers.UpSampling3D(size=(1, 2, 2))(bn12)
    up7 = layers.concatenate([up_conv6, conv3], axis=concat_axis)
    conv7 = layers.ConvLSTM2D(128, (3, 3), activation='relu', padding='same', recurrent_activation = "hard_sigmoid",return_sequences=True)(up7)
    bn13 = BatchNormalization(axis=2)(conv7)
    conv7 = layers.ConvLSTM2D(128, (3, 3), activation='relu', padding='same', recurrent_activation = "hard_sigmoid",return_sequences=True)(bn13)
    bn14 = BatchNormalization(axis=2)(conv7)

    up_conv7 = layers.UpSampling3D(size=(1, 2, 2))(bn14)
    up8 = layers.concatenate([up_conv7, conv2], axis=concat_axis)
    conv8 = layers.ConvLSTM2D(64, (3, 3), activation='relu', padding='same', recurrent_activation = "hard_sigmoid",return_sequences=True)(up8)
    bn15 = BatchNormalization(axis=2)(conv8)
    conv8 = layers.ConvLSTM2D(64, (3, 3), activation='relu', padding='same', recurrent_activation = "hard_sigmoid",return_sequences=True)(bn15)
    bn16 = BatchNormalization(axis=2)(conv8)

    up_conv8 = layers.UpSampling3D(size=(1, 2, 2))(bn16)
    up9 = layers.concatenate([up_conv8, conv1], axis=concat_axis)
    conv9 = layers.ConvLSTM2D(32, (3, 3), activation='relu', padding='same', recurrent_activation = "hard_sigmoid",return_sequences=True)(up9)
    bn17 = BatchNormalization(axis=2)(conv9)
    conv9 = layers.ConvLSTM2D(32, (3, 3), activation='relu', padding='same', recurrent_activation = "hard_sigmoid",return_sequences=True)(bn17)
    bn18 = BatchNormalization(axis=2)(conv9)

    conv10 = layers.ConvLSTM2D(1, (1, 1), recurrent_activation = "hard_sigmoid",return_sequences=True)(bn18)
    #bn19 = BatchNormalization(axis=2)(conv10)

    model = models.Model(inputs=inputs, outputs=conv10)

    sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='mae', optimizer=sgd, metrics=['mse','acc'])
    #model.compile(loss='mae', optimizer=Adam(lr=0.01), metrics=['mse'])
    print(model.summary())

    return model

In [3]:
def get_unet_LSTM2():
    #concat_axis = 3 # 3 o 1
    concat_axis = 2 
    #inputs = layers.Input(shape = (80, 120, 3))
    inputs = layers.Input(shape = (4, 3, 128, 144))
    
    #encoder
    bn0 = BatchNormalization(axis=2)(inputs)
    conv1 = layers.Conv2D(32, (3, 3), activation='relu', padding='same', name='conv1_1')(bn0)
    bn1 = BatchNormalization(axis=2)(conv1)
    conv1 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(bn1)
    bn2 = BatchNormalization(axis=2)(conv1)
    pool1 = layers.MaxPooling3D(pool_size=(1,2, 2))(bn2)
    conv2 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(pool1)
    bn3 = BatchNormalization(axis=2)(conv2)
    conv2 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(bn3)
    bn4 = BatchNormalization(axis=2)(conv2)
    pool2 = layers.MaxPooling3D(pool_size=(1,2, 2))(bn4)

    conv3 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(pool2)
    bn5 = BatchNormalization(axis=2)(conv3)
    conv3 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(bn5)
    bn6 = BatchNormalization(axis=2)(conv3)
    pool3 = layers.MaxPooling3D(pool_size=(1,2, 2))(bn6)

    conv4 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(pool3)
    bn7 = BatchNormalization(axis=2)(conv4)
    conv4 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(bn7)
    bn8 = BatchNormalization(axis=2)(conv4)
    #pool4 = layers.MaxPooling2D(pool_size=(2, 3))(bn8)
    pool4 = layers.MaxPooling3D(pool_size=(1,2, 2))(bn8)
    
    conv5 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(pool4)
    bn9 = BatchNormalization(axis=2)(conv5)
    conv5 = layers.Conv2D(512, (3, 3), activation='relu', padding='same')(bn9)
    bn10 = BatchNormalization(axis=2)(conv5)

    ##decoder
    #up_conv5 = layers.UpSampling2D(size=(2, 3))(bn10)
    up_conv5 = layers.UpSampling3D(size=(1,2, 2))(bn10)
    up6 = layers.concatenate([up_conv5, conv4], axis=concat_axis)
    conv6 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(up6)
    bn11 = BatchNormalization(axis=2)(conv6)
    conv6 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(bn11)
    bn12 = BatchNormalization(axis=2)(conv6)

    up_conv6 = layers.UpSampling3D(size=(1,2, 2))(bn12)
    up7 = layers.concatenate([up_conv6, conv3], axis=concat_axis)
    conv7 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(up7)
    bn13 = BatchNormalization(axis=2)(conv7)
    conv7 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(bn13)
    bn14 = BatchNormalization(axis=2)(conv7)

    up_conv7 = layers.UpSampling3D(size=(1,2, 2))(bn14)
    up8 = layers.concatenate([up_conv7, conv2], axis=concat_axis)
    conv8 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(up8)
    bn15 = BatchNormalization(axis=2)(conv8)
    conv8 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(bn15)
    bn16 = BatchNormalization(axis=2)(conv8)

    up_conv8 = layers.UpSampling3D(size=(1,2, 2))(bn16)
    up9 = layers.concatenate([up_conv8, conv1], axis=concat_axis)
    conv9 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(up9)
    bn17 = BatchNormalization(axis=2)(conv9)
    conv9 = layers.Conv2D(32, (3, 3), activation='relu', padding='same')(bn17)
    bn18 = BatchNormalization(axis=2)(conv9)

    conv10 = layers.ConvLSTM2D(1, (1, 1), recurrent_activation = "hard_sigmoid",return_sequences=True)(bn18)
    #bn19 = BatchNormalization(axis=2)(conv10)

    model = models.Model(inputs=inputs, outputs=conv10)

    sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
    model.compile(loss='mae', optimizer=sgd, metrics=['mse','acc'])
    #model.compile(loss='mae', optimizer=Adam(lr=0.01), metrics=['mse'])
    print(model.summary())

    return model

In [4]:
model = get_unet_LSTM2()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 4, 3, 128, 1 0                                            
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 4, 3, 128, 14 12          input_1[0][0]                    
__________________________________________________________________________________________________
conv1_1 (Conv2D)                (None, 4, 32, 128, 1 896         batch_normalization[0][0]        
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 4, 32, 128, 1 128         conv1_1[0][0]                    
______________________________________________________________________________________________

In [5]:
## X:
X_train = np.load("/opt/datos/dataset/gfs/X_train_3-7-9.npy")
X_val   = np.load("/opt/datos/dataset/gfs/X_val_3-7-9.npy")
print(X_train.shape)
print(X_val.shape)

## Y:
Y_train = np.load("/opt/datos/dataset/gfs/Y_train.npy")
Y_val   = np.load("/opt/datos/dataset/gfs/Y_val.npy")
print(Y_train.shape)
print(Y_val.shape)

(13502, 3, 137, 157)
(2893, 3, 137, 157)
(13502, 137, 157)
(2893, 137, 157)


In [6]:
X_train_t = np.reshape(X_train[0:13500],(3375,4,3,137,157))
X_val_t   = np.reshape(X_val[0:2892],(723,4,3,137,157))
X_train = None
X_val = None

Y_train_t = np.reshape(Y_train[0:13500],(3375,4,137,157))
Y_val_t   = np.reshape(Y_val[0:2892],(723,4,137,157))
Y_train = None
Y_val = None

In [7]:
# Recorte para obtener 128x144
X_train_t = X_train_t[:, :, :, 0:128, 0:144]
X_val_t = X_val_t[:, :, :, 0:128, 0:144]
print(X_train_t.shape)
print(X_val_t.shape)

Y_train_t = Y_train_t[:, :, 0:128, 0:144]
Y_val_t = Y_val_t[:, :, 0:128, 0:144]
print(Y_train_t.shape)
print(Y_val_t.shape)

(3375, 4, 3, 128, 144)
(723, 4, 3, 128, 144)
(3375, 4, 128, 144)
(723, 4, 128, 144)


In [8]:
Y_train_t = np.expand_dims(Y_train_t, axis=2)
Y_val_t = np.expand_dims(Y_val_t, axis=2)

print(Y_train_t.shape)
print(Y_val_t.shape)

(3375, 4, 1, 128, 144)
(723, 4, 1, 128, 144)


In [9]:
history = model.fit(X_train_t, Y_train_t, epochs=20, verbose=1, validation_data=(X_val_t, Y_val_t))

Epoch 1/20
106/106 [==============================] - 397s 3s/step - loss: 0.5194 - mse: 3.6423 - acc: 0.0124 - val_loss: 0.3734 - val_mse: 2.4607 - val_acc: 0.0102
Epoch 2/20
106/106 [==============================] - 268s 3s/step - loss: 0.4401 - mse: 3.6534 - acc: 0.0354 - val_loss: 0.3702 - val_mse: 2.4426 - val_acc: 0.0124
Epoch 3/20
106/106 [==============================] - 267s 3s/step - loss: 0.4367 - mse: 3.5945 - acc: 0.0423 - val_loss: 0.3674 - val_mse: 2.4196 - val_acc: 0.0306
Epoch 4/20
106/106 [==============================] - 267s 3s/step - loss: 0.4293 - mse: 3.5235 - acc: 0.0462 - val_loss: 0.3622 - val_mse: 2.3282 - val_acc: 0.0544
Epoch 5/20
106/106 [==============================] - 267s 3s/step - loss: 0.4261 - mse: 3.4321 - acc: 0.0474 - val_loss: 0.3568 - val_mse: 2.2673 - val_acc: 0.0583
Epoch 6/20
106/106 [==============================] - 267s 3s/step - loss: 0.4313 - mse: 3.5733 - acc: 0.0503 - val_loss: 0.3553 - val_mse: 2.2084 - val_acc: 0.0613
Epoch 7/20

In [ ]:
with open('/opt/datos/entrenamientos/gfs/08', 'wb') as file_pi:
    pickle.dump(history.history, file_pi)

In [ ]:
history = pickle.load( open( "/opt/datos/entrenamientos/gfs/08", "rb" ) )

In [ ]:
print("~ Unet Dataset GFS 2015-2021 ~")
print(f"val_loss: {history['val_loss'][-1]}")
print(f"loss: {history['loss'][-1]}")
#print(history)
plt.plot(history['loss'])
plt.plot(history['val_loss'])
plt.title('Unet network training 1000-700-500 hPa')
plt.ylabel('mean absolute error')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

In [ ]:
model.save('/opt/datos/entrenamientos/gfs/08.h5')